In [1]:
!pip install pytorch-lightning


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 41.5 MB/s eta 0:00:00


In [2]:
import os
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import random

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split

import torchvision
from torchvision import transforms, datasets

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, TQDMProgressBar
from pytorch_lightning.loggers import TensorBoardLogger

from torchmetrics import Accuracy, F1Score, AUROC, MetricCollection

### Зафиксированы random_state

In [3]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed(42)

Random seed set as 42


In [6]:
# Проверяем, доступен ли GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


Создаем класс FashionMNISTDataModule, реализуем в нем:
- загрузку данных,
- предобработку (перевод в тензоры, нормализация, etc)
- разбиение на train/val/test части
- создание dataloader'ов

In [7]:
class FashionMNISTDataModule(pl.LightningDataModule):
    def __init__(self, data_dir="./data", batch_size=64):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

    def prepare_data(self):
        datasets.FashionMNIST(self.data_dir, train=True, download=True)
        datasets.FashionMNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        if stage == "fit" or stage is None:
            full_train = datasets.FashionMNIST(self.data_dir, train=True, transform=self.transform)
            self.train_set, self.val_set = random_split(full_train, [50000, 10000])
        if stage == "test" or stage is None:
            self.test_set = datasets.FashionMNIST(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)


 Создаем класс модели FashionMNIST (наследник LightningModule) и реализуем в нем:
- training_step, validation_step, test_step
- расчет метрик на валидации и тестировании из TorchMetrics: F1, ROC AUC
- логирование метрик и функций потерь на каждой эпохе валидации/теста
- подходящие optimizer и lr-scheduler, а также их гиперпараметры

In [8]:
class FashionMNISTModel(pl.LightningModule):
    def __init__(self, learning_rate=1e-3):
        super().__init__()
        self.learning_rate = learning_rate
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
        self.criterion = nn.CrossEntropyLoss()
        metrics = MetricCollection({
            "accuracy": Accuracy(task="multiclass", num_classes=10, average="macro"),
            "f1": F1Score(task="multiclass", num_classes=10, average="macro"),
            "roc_auc": AUROC(task="multiclass", num_classes=10, average="macro")
        })
        self.train_metrics = metrics.clone(prefix="train_")
        self.val_metrics = metrics.clone(prefix="val_")
        self.test_metrics = metrics.clone(prefix="test_")

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = self.criterion(logits, y)
        probs = torch.softmax(logits, dim=1)
        self.log_dict(self.train_metrics(probs, y))
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = self.criterion(logits, y)
        probs = torch.softmax(logits, dim=1)
        self.log_dict(self.val_metrics(probs, y))
        self.log("val_loss", loss, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = self.criterion(logits, y)
        probs = torch.softmax(logits, dim=1)
        self.log_dict(self.test_metrics(probs, y))
        self.log("test_loss", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
        return [optimizer], [scheduler]

Обучаем модель с помощью trainer'а:
- добавляем EarlyStopping
- реализуем визуализацию логов через tensorboard
- проверяем качество на тестовой части данных

In [9]:
data_module = FashionMNISTDataModule(batch_size=64)

In [10]:
model = FashionMNISTModel(learning_rate=1e-3)

In [11]:
early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")
checkpoint_callback = ModelCheckpoint(dirpath="checkpoints", save_top_k=1, monitor="val_loss", mode="min")

In [12]:
logger = TensorBoardLogger("logs", name="fashion_mnist")

In [13]:
trainer = pl.Trainer(
    max_epochs=20,
    logger=logger,
    callbacks=[early_stopping, checkpoint_callback, TQDMProgressBar(refresh_rate=20)]
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [14]:
trainer.fit(model, data_module)

100%|██████████| 26.4M/26.4M [00:02<00:00, 11.7MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 206kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.71MB/s]


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [15]:
trainer.test(model, data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8833662867546082     │
│          test_f1          │    0.8744624853134155     │
│         test_loss         │    0.3485661745071411     │
│       test_roc_auc        │    0.9894993901252747     │
└───────────────────────────┴───────────────────────────┘

[{'test_accuracy': 0.8833662867546082,
  'test_f1': 0.8744624853134155,
  'test_roc_auc': 0.9894993901252747,
  'test_loss': 0.3485661745071411}]